# everyrow SDK Basic Usage

This notebook demonstrates all the core operations in the everyrow SDK:

1. **Screen** - Filter rows based on criteria that need judgment
2. **Rank** - Score rows by qualitative factors
3. **Dedupe** - Deduplicate when fuzzy matching fails
4. **Merge** - Join tables when keys don't match exactly
5. **Derive** - Add computed columns (no AI needed)
6. **Single Agent** - Web research on a single input
7. **Agent Map** - Web research on every row of a dataframe

Get an API key at [everyrow.io/api-key](https://everyrow.io/api-key) to run this notebook.

In [27]:
import os
from textwrap import dedent

from dotenv import load_dotenv
from pandas import DataFrame
from pydantic import BaseModel, Field

from everyrow.session import create_session

load_dotenv()

True

## 1. Screen

Filter rows based on criteria you can't put in a WHERE clause. This example performs vendor risk assessment - evaluating security track records and financial stability requires judgment, not pattern matching.

In [28]:
from everyrow.ops import screen


vendors = DataFrame(
    [
        {"company": "Okta", "category": "Identity Management", "website": "okta.com"},
        {"company": "LastPass", "category": "Password Management", "website": "lastpass.com"},
        {"company": "Snowflake", "category": "Data Warehouse", "website": "snowflake.com"},
        {"company": "Cloudflare", "category": "CDN & Security", "website": "cloudflare.com"},
        {"company": "MongoDB", "category": "Database", "website": "mongodb.com"},
    ]
)

print("Input vendors:")
print(vendors.to_string())

Input vendors:
      company             category         website
0        Okta  Identity Management        okta.com
1    LastPass  Password Management    lastpass.com
2   Snowflake       Data Warehouse   snowflake.com
3  Cloudflare       CDN & Security  cloudflare.com
4     MongoDB             Database     mongodb.com


In [29]:
# Basic screen - returns only passes/fails boolean
basic_screen_result = await screen(
    task=dedent("""
        Perform vendor risk assessment for each company. Research and evaluate:

        1. Security track record: Have they had any significant data breaches or security
        incidents in the past 3 years? How did they respond?

        2. Financial stability: Are there signs of financial distress (major layoffs,
        funding difficulties, declining revenue)?

        3. Overall recommendation: Based on your research, should we proceed with
        this vendor for enterprise use?

        Only approve vendors with low or medium risk and no unresolved critical security incidents.
    """),
    input=vendors,
)

print("Basic Screen Results (passes/fails only):")
print(basic_screen_result.data.to_string())

TaskResultResponse(task_id=UUID('5837e0b8-4109-431b-a095-a21e9bf4b565'), status=<TaskStatus.COMPLETED: 'completed'>, artifact_id=UUID('2892391d-26de-44d3-9f5f-07ccd88074d0'), data=[TaskResultResponseDataType0Item(additional_properties={'company': 'Snowflake', 'category': 'Data Warehouse', 'website': 'snowflake.com', 'research': {'screening_result': "Snowflake was the target of a large-scale credential-stuffing campaign in May 2024 that affected over 160 customers [dd5dc7]. While not a breach of Snowflake's own infrastructure, the company responded by implementing mandatory MFA options and extensive hardening guidance [dd5dc7]. Financials remain strong with 28-29% revenue growth and high retention rates, despite slowing growth compared to previous years. Deductions: The incident was a configuration issue rather than a core platform breach, and the response was proactive, leading to a Medium risk.", 'passes': "Snowflake was the target of a large-scale credential-stuffing campaign in May 

In [ ]:
# Screen with response_model for additional structured context
class VendorRiskAssessment(BaseModel):
    passes: bool = Field(description="Whether the vendor passes risk assessment")
    risk_level: str = Field(description="Risk level: Low, Medium, or High")
    security_summary: str = Field(description="Brief summary of security track record")
    financial_summary: str = Field(description="Brief summary of financial stability")


detailed_screen_result = await screen(
    task=dedent("""
        Perform vendor risk assessment for each company. Research and evaluate:

        1. Security track record: Have they had any significant data breaches or security
        incidents in the past 3 years? How did they respond?

        2. Financial stability: Are there signs of financial distress (major layoffs,
        funding difficulties, declining revenue)?

        3. Overall recommendation: Based on your research, should we proceed with
        this vendor for enterprise use?

        Only approve vendors with low or medium risk and no unresolved critical security incidents.
    """),
    input=vendors,
    response_model=VendorRiskAssessment,
)

print("Detailed Screen Results (with structured context):")
print(detailed_screen_result.data.to_string())

TaskResultResponse(task_id=UUID('713adf55-a975-4345-aab3-59f78bc65adf'), status=<TaskStatus.COMPLETED: 'completed'>, artifact_id=UUID('d0b17291-1e52-4350-a6ce-f004ca9ed9ff'), data=[TaskResultResponseDataType0Item(additional_properties={'company': 'Snowflake', 'category': 'Data Warehouse', 'website': 'snowflake.com', 'research': {'screening_result': "Snowflake was at the center of a massive security event in mid-2024 where over 160 customer environments (including AT&T and Ticketmaster) were breached. While Snowflake's core systems were not compromised, the lack of mandated MFA for customer accounts was a critical factor. Snowflake has since responded by mandating MFA for all users. Financially, Snowflake shows strong revenue growth (~$3.4B in FY2025, up 29%) and high net revenue retention (125-127%), though it continues to report substantial GAAP net losses (~$1B annually). The 2024 incident is considered resolved through policy changes (mandatory MFA), and the financial losses are typ

## 2. Rank

Score rows by things you can't put in a database field. This example ranks AI research organizations by leadership citation counts - information that requires researching each org's leaders and their publications.

In [31]:
from everyrow.ops import rank

ai_research_orgs = DataFrame(
    [
        {"organization": "OpenAI", "type": "Private lab", "founded": 2015},
        {"organization": "Google DeepMind", "type": "Corporate lab", "founded": 2010},
        {"organization": "Anthropic", "type": "Private lab", "founded": 2021},
        {"organization": "Meta FAIR", "type": "Corporate lab", "founded": 2013},
        {"organization": "Microsoft Research", "type": "Corporate lab", "founded": 1991},
        {"organization": "Stanford HAI", "type": "Academic", "founded": 2019},
    ]
)

print("AI Research Organizations:")
print(ai_research_orgs.to_string())

AI Research Organizations:
         organization           type  founded
0              OpenAI    Private lab     2015
1     Google DeepMind  Corporate lab     2010
2           Anthropic    Private lab     2021
3           Meta FAIR  Corporate lab     2013
4  Microsoft Research  Corporate lab     1991
5        Stanford HAI       Academic     2019


In [32]:
rank_task = dedent("""
    Research the total citation count of all leaders of the given AI research organization.

    A leader is defined as a C-Suite or founder of the company.
    Citation count should count all major publications. Top ten by each person is sufficient.
""")

# Basic ranking - returns only the score field
basic_rank_result = await rank(
    task=rank_task,
    input=ai_research_orgs,
    field_name="contribution_score",
    field_type="int",
    ascending_order=False,
)

print("Basic Rankings (score only):")
print(basic_rank_result.data.to_string())

TaskResultResponse(task_id=UUID('323f344b-c42d-4dab-919d-850199ddec9f'), status=<TaskStatus.COMPLETED: 'completed'>, artifact_id=UUID('3283dc61-70ec-4fde-adc2-f3a0f83b9fd4'), data=[TaskResultResponseDataType0Item(additional_properties={'organization': 'OpenAI', 'type': 'Private lab', 'founded': 2015, 'research': {'contribution_score': "OpenAI's leadership and founding team (11 co-founders) were identified from official founding announcements and Wikipedia [2, 16, 22]. Citations were collected from Google Scholar profiles. Key leaders include Ilya Sutskever (747,609 citations) [13], Diederik (Durk) Kingma (341,972 citations) [24], John Schulman (161,754 citations) [15], Wojciech Zaremba (124,058 citations) [14], Mark Chen (140,160 citations) [12], and Andrej Karpathy (77,898 citations) [16]. Other founders like Sam Altman, Greg Brockman, and Elon Musk have low or non-academic citation profiles in AI research (Altman/Brockman ~0-5k; Musk ~4.5k) [4, 5, 22]. Top 10 publications were identi

In [33]:
# Ranking with response_model for additional structured context
class ContributionRanking(BaseModel):
    contribution_score: int = Field(description="Total citation count")
    most_significant_contribution: str = Field(
        description="Single most important paper authored by a firm leader"
    )


detailed_rank_result = await rank(
    task=rank_task + "\n\nAlso identify their single most significant contribution.",
    input=ai_research_orgs,
    field_name="contribution_score",
    response_model=ContributionRanking,
    ascending_order=False,
)

print("Detailed Rankings (with structured context):")
print(detailed_rank_result.data.to_string())

TaskResultResponse(task_id=UUID('bdb71be7-42e1-4064-9d4a-51e63dc02e93'), status=<TaskStatus.COMPLETED: 'completed'>, artifact_id=UUID('c2057ef4-fca7-41ba-a846-f25a4783274f'), data=[TaskResultResponseDataType0Item(additional_properties={'organization': 'OpenAI', 'type': 'Private lab', 'founded': 2015, 'contribution_score': 1610443, 'research': {'most_significant_contribution': 'The collective contribution of the founding team was the creation of the GPT architecture and the transition of AI research from academic models to massive-scale production systems. Individually, Sutskever is cited for Sequence to Sequence learning, Kingma for Adam and VAEs, and Altman for the strategic scaling of the organization.', 'contribution_score': 'Calculated by summing the total Google Scholar citation counts of key current and former C-Suite/founders: Ilya Sutskever (747,609), Diederik P. Kingma (341,972), John Schulman (161,754), Mark Chen (140,160), Wojciech Zaremba (124,058), Andrej Karpathy (77,898)

## 3. Dedupe

Deduplicate when fuzzy matching falls short. This example deduplicates academic papers where the same paper may appear with different identifiers (arXiv ID vs DOI), different title formats, or as preprint vs published versions.

In [34]:
from everyrow.ops import dedupe

papers = DataFrame(
    [
        {
            "title": "Attention Is All You Need",
            "authors": "Vaswani et al.",
            "venue": "NeurIPS 2017",
            "identifier": "10.5555/3295222.3295349",
        },
        {
            "title": "Attention Is All You Need",
            "authors": "Vaswani, Shazeer, Parmar et al.",
            "venue": "arXiv",
            "identifier": "1706.03762",
        },
        {
            "title": "BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding",
            "authors": "Devlin et al.",
            "venue": "NAACL 2019",
            "identifier": "10.18653/v1/N19-1423",
        },
        {
            "title": "BERT: Pre-training of Deep Bidirectional Transformers",
            "authors": "Devlin, Chang, Lee, Toutanova",
            "venue": "arXiv",
            "identifier": "1810.04805",
        },
        {
            "title": "Language Models are Few-Shot Learners",
            "authors": "Brown et al.",
            "venue": "NeurIPS 2020",
            "identifier": "GPT-3",
        },
        {
            "title": "GPT-3: Language Models are Few-Shot Learners",
            "authors": "Brown, Mann, Ryder et al.",
            "venue": "arXiv",
            "identifier": "2005.14165",
        },
        {
            "title": "LLaMA: Open and Efficient Foundation Language Models",
            "authors": "Touvron et al.",
            "venue": "arXiv",
            "identifier": "2302.13971",
        },
        {
            "title": "Llama 2: Open Foundation and Fine-Tuned Chat Models",
            "authors": "Touvron et al.",
            "venue": "arXiv",
            "identifier": "2307.09288",
        },
    ]
)

print(f"Input papers ({len(papers)} rows):")
print(papers.to_string())

Input papers (8 rows):
                                                                              title                          authors         venue               identifier
0                                                         Attention Is All You Need                   Vaswani et al.  NeurIPS 2017  10.5555/3295222.3295349
1                                                         Attention Is All You Need  Vaswani, Shazeer, Parmar et al.         arXiv               1706.03762
2  BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding                    Devlin et al.    NAACL 2019     10.18653/v1/N19-1423
3                             BERT: Pre-training of Deep Bidirectional Transformers    Devlin, Chang, Lee, Toutanova         arXiv               1810.04805
4                                             Language Models are Few-Shot Learners                     Brown et al.  NeurIPS 2020                    GPT-3
5                                      GP

In [35]:
dedupe_result = await dedupe(
    input=papers,
    equivalence_relation=dedent("""
        Two entries are duplicates if they represent the same research work, which requires
        verifying through research:

        - An arXiv preprint and its published conference/journal version are duplicates
        - Papers with slightly different titles but same core contribution are duplicates
        - Different author list formats (et al. vs full list) don't matter
        - Papers with different identifiers (arXiv ID vs DOI) may still be duplicates

        However, genuinely different papers (e.g., LLaMA 1 vs LLaMA 2) are NOT duplicates,
        even if authors and topics overlap. Research each paper to determine if they
        report the same findings or are distinct works.
    """),
)

print("Deduplicated Paper List:")
print(dedupe_result.data.to_string())
print(f"\nOriginal entries: {len(papers)}")
print(f"Unique papers: {len(dedupe_result.data)}")
print(f"Duplicates removed: {len(papers) - len(dedupe_result.data)}")

TaskResultResponse(task_id=UUID('8e4d15d0-b264-4677-b17d-80f08e02680b'), status=<TaskStatus.COMPLETED: 'completed'>, artifact_id=UUID('7ec69b9f-9b71-4953-90d7-af4d0ed079ab'), data=[TaskResultResponseDataType0Item(additional_properties={'title': 'Attention Is All You Need', 'authors': 'Vaswani et al.', 'venue': 'NeurIPS 2017', 'identifier': '10.5555/3295222.3295349', 'equivalence_class_id': '19fa4363-4c9a-43e9-a3f5-f7dfaf3c8c9c', 'selected': True, 'equivalence_class_name': 'Attention Is All You Need'}), TaskResultResponseDataType0Item(additional_properties={'title': 'Attention Is All You Need', 'authors': 'Vaswani, Shazeer, Parmar et al.', 'venue': 'arXiv', 'identifier': '1706.03762', 'equivalence_class_id': '19fa4363-4c9a-43e9-a3f5-f7dfaf3c8c9c', 'selected': False, 'equivalence_class_name': 'Attention Is All You Need'}), TaskResultResponseDataType0Item(additional_properties={'title': 'BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding', 'authors': 'Devlin 

## 4. Merge

Join two tables when the keys don't match exactly. This example merges clinical trial data with pharmaceutical company information - the challenge is that trial sponsors are often subsidiaries or use abbreviated names (e.g., "MSD" instead of "Merck").

In [36]:
from everyrow.ops import merge

clinical_trials = DataFrame(
    [
        {
            "trial_id": "NCT05432109",
            "sponsor": "Genentech",
            "indication": "Non-small cell lung cancer",
            "phase": "Phase 3",
        },
        {
            "trial_id": "NCT05891234",
            "sponsor": "Janssen Pharmaceuticals",
            "indication": "Multiple myeloma",
            "phase": "Phase 2",
        },
        {
            "trial_id": "NCT05567890",
            "sponsor": "MSD",
            "indication": "Melanoma",
            "phase": "Phase 3",
        },
        {
            "trial_id": "NCT05234567",
            "sponsor": "AbbVie Inc",
            "indication": "Rheumatoid arthritis",
            "phase": "Phase 3",
        },
        {
            "trial_id": "NCT05678901",
            "sponsor": "BMS",
            "indication": "Acute myeloid leukemia",
            "phase": "Phase 2",
        },
    ]
)

pharma_companies = DataFrame(
    [
        {
            "company": "Roche Holding AG",
            "hq_country": "Switzerland",
            "2024_revenue_billions": 58.7,
        },
        {
            "company": "Johnson & Johnson",
            "hq_country": "United States",
            "2024_revenue_billions": 85.2,
        },
        {
            "company": "Merck & Co.",
            "hq_country": "United States",
            "2024_revenue_billions": 60.1,
        },
        {
            "company": "AbbVie",
            "hq_country": "United States",
            "2024_revenue_billions": 56.3,
        },
        {
            "company": "Bristol-Myers Squibb",
            "hq_country": "United States",
            "2024_revenue_billions": 45.0,
        },
    ]
)

print("Clinical Trials:")
print(clinical_trials.to_string())
print("\nPharma Companies:")
print(pharma_companies.to_string())

Clinical Trials:
      trial_id                  sponsor                  indication    phase
0  NCT05432109                Genentech  Non-small cell lung cancer  Phase 3
1  NCT05891234  Janssen Pharmaceuticals            Multiple myeloma  Phase 2
2  NCT05567890                      MSD                    Melanoma  Phase 3
3  NCT05234567               AbbVie Inc        Rheumatoid arthritis  Phase 3
4  NCT05678901                      BMS      Acute myeloid leukemia  Phase 2

Pharma Companies:
                company     hq_country  2024_revenue_billions
0      Roche Holding AG    Switzerland                   58.7
1     Johnson & Johnson  United States                   85.2
2           Merck & Co.  United States                   60.1
3                AbbVie  United States                   56.3
4  Bristol-Myers Squibb  United States                   45.0


In [37]:
merge_result = await merge(
    task=dedent("""
        Merge clinical trial data with parent pharmaceutical company information.

        The sponsor names in the trials table are often subsidiaries or abbreviations:
        - Research which parent company owns each trial sponsor
        - Match trials to their parent company's financial data

        For example, Genentech is a subsidiary of Roche, Janssen is part of J&J,
        MSD is Merck's name outside the US, BMS is Bristol-Myers Squibb.
    """),
    left_table=clinical_trials,
    right_table=pharma_companies,
    merge_on_left="sponsor",
    merge_on_right="company",
)

print("Clinical Trials with Parent Company Data:")
print(merge_result.data.to_string())

TaskResultResponse(task_id=UUID('28c2d5b9-a026-4530-bf2d-62f164032be3'), status=<TaskStatus.COMPLETED: 'completed'>, artifact_id=UUID('d34ab97c-989c-4712-8e5c-dcb335431df5'), data=[TaskResultResponseDataType0Item(additional_properties={'trial_id': 'NCT05432109', 'sponsor': 'Genentech', 'indication': 'Non-small cell lung cancer', 'phase': 'Phase 3', 'company': 'Roche Holding AG', 'hq_country': 'Switzerland', '2024_revenue_billions': 58.7, 'research': {'company': 'This row was matched due to the information in both tables', 'hq_country': 'This row was matched due to the information in both tables', '2024_revenue_billions': 'This row was matched due to the information in both tables'}}), TaskResultResponseDataType0Item(additional_properties={'trial_id': 'NCT05891234', 'sponsor': 'Janssen Pharmaceuticals', 'indication': 'Multiple myeloma', 'phase': 'Phase 2', 'company': 'Johnson & Johnson', 'hq_country': 'United States', '2024_revenue_billions': 85.2, 'research': {'company': 'This row was 

## 6. Single Agent

Run web research on a single input. Agents can generate tabular data or answer questions based on research. This example first generates a competitor dataset, then analyzes it for market gaps.

In [38]:
from everyrow.ops import single_agent


class Competitor(BaseModel):
    company: str = Field(description="Company name")
    pricing_tier: str = Field(description="Pricing model, e.g. 'Freemium, $10-50/user/mo'")
    target_market: str = Field(description="Primary customer segment")
    key_features: str = Field(description="Top 3 features or differentiators")


# Step 1: Generate a dataset of competitors
print("Step 1: Research competitors")
competitors = await single_agent(
    task="Find the top 10 competitors in the B2B expense management software market",
    response_model=Competitor,
    return_table=True,
)
print(competitors.data.to_string())

Step 1: Research competitors
TaskResultResponse(task_id=UUID('a98a88f6-7486-4594-9f60-36166a3317de'), status=<TaskStatus.COMPLETED: 'completed'>, artifact_id=UUID('3d34a1e4-dad7-46d1-8a8d-e6c8d19ff46f'), data=[TaskResultResponseDataType0Item(additional_properties={'company': 'SAP Concur', 'pricing_tier': 'Quote-based, varies by modules and user count', 'target_market': 'Large enterprises and mid-market companies', 'key_features': 'Integrated travel and expense, invoice management, robust reporting', 'research': {'company': 'Company name is widely known and consistently listed as a top competitor in expense management.', 'pricing_tier': 'Pricing is not publicly available and requires direct contact for a quote, as is common for enterprise-level solutions. This is a deduction based on the absence of public pricing on their website and common industry practices for large-scale software.', 'target_market': "SAP Concur's solutions are comprehensive and cater to the complex needs of larger o

In [39]:
# Step 2: Distill insights from the dataset
print("Step 2: Identify market gaps")
insights = await single_agent(
    task="""
        What gaps exist in the B2B expense management software market
        that these competitors aren't addressing?
    """,
    input=competitors,
)
print(insights.data.answer)

Step 2: Identify market gaps
TaskResultResponse(task_id=UUID('554ae8d7-745b-4cfc-9d19-cded29430c79'), status=<TaskStatus.COMPLETED: 'completed'>, artifact_id=UUID('9b5f29d0-e0e7-40f9-b9ca-c90bd20965da'), data=TaskResultResponseDataType1(additional_properties={'answer': 'The B2B expense management software market, as represented by the provided competitors, shows gaps in highly specialized industry-specific solutions, advanced predictive analytics for cost optimization beyond basic savings insights, and comprehensive, integrated global compliance and tax management for multinational operations. While some offer travel integration or general expense management, none explicitly highlight deep customization for niche industries (e.g., construction project-based expenses, healthcare regulatory compliance), sophisticated AI-driven forecasting of future spend, or a unified platform for managing diverse global tax regulations and compliance standards across multiple jurisdictions.', 'research'

## 7. Agent Map

Run web research on every row of a dataframe. This example researches financial information for tech companies - information that requires looking up each company individually.

In [40]:
from everyrow.ops import agent_map


class CompanyFinancials(BaseModel):
    annual_revenue_usd: int = Field(description="Most recent annual revenue in USD")
    employee_count: int = Field(description="Current number of employees")
    founded_year: int = Field(description="Year the company was founded")


companies = DataFrame(
    [
        {"company": "Stripe", "industry": "Payments"},
        {"company": "Databricks", "industry": "Data & AI"},
        {"company": "Canva", "industry": "Design"},
        {"company": "Figma", "industry": "Design"},
        {"company": "Notion", "industry": "Productivity"},
    ]
)

print("Companies to research:")
print(companies.to_string())

Companies to research:
      company      industry
0      Stripe      Payments
1  Databricks     Data & AI
2       Canva        Design
3       Figma        Design
4      Notion  Productivity


In [41]:
# Basic usage with default response
basic_result = await agent_map(
    task="Find the company's most recent annual revenue in USD",
    input=companies,
)

print("Basic Results:")
print(basic_result.data.to_string())

TaskResultResponse(task_id=UUID('3dbc4802-b3da-42b1-a4b2-6afcafae6fb1'), status=<TaskStatus.COMPLETED: 'completed'>, artifact_id=UUID('9c579239-298d-4d7f-bd0b-f3c11a4d3808'), data=[TaskResultResponseDataType0Item(additional_properties={'company': 'Stripe', 'industry': 'Payments', 'answer': "Stripe's most recent annual revenue is estimated to be $14.4 billion for 2022.", 'research': {'answer': 'Stripe is a private company and does not publicly disclose its financial results. Therefore, its revenue figures are estimates from various financial news outlets. The $14.4 billion figure for 2022 is cited by Forbes, which attributes the information to \'people familiar with the company\'s financials\'. Other sources, such as Bloomberg, also reported similar figures for 2022, indicating a consensus among financial reporters. I have chosen the Forbes estimate as it is a reputable financial publication.\n\nForbes: "Stripe Revenue Hit $14.4 Billion In 2022, Up From $12 Billion In 2021" https://www.

In [42]:
# Structured output with a response model
structured_result = await agent_map(
    task=dedent("""
        Research the company's financials. Find:
        1. Their most recent annual revenue (in USD)
        2. Current employee count
        3. Year founded

        If the company is a subsidiary, report figures for the subsidiary
        specifically, not the parent company.
    """),
    input=companies,
    response_model=CompanyFinancials,
)

print("Structured Results:")
print(structured_result.data.to_string())

TaskResultResponse(task_id=UUID('462cf57e-b7b0-4b1f-84cd-4923ba455967'), status=<TaskStatus.COMPLETED: 'completed'>, artifact_id=UUID('ba2c572f-c3a0-4d07-91c8-de53181d30a7'), data=[TaskResultResponseDataType0Item(additional_properties={'company': 'Stripe', 'industry': 'Payments', 'annual_revenue_usd': 14000000000, 'employee_count': 8000, 'founded_year': 2010, 'research': {'annual_revenue_usd': "Stripe's annual revenue for 2022 was reported as $14 billion. This figure is cited by multiple sources, including Forbes and Business Insider, based on company statements. The most recent full year for which data is widely available is 2022. Source: https://www.forbes.com/sites/alexkonrad/2023/03/15/stripe-valuation-down-to-50-billion-after-tender-offer/?sh=211111111111, https://www.businessinsider.com/stripe-revenue-profit-valuation-ipo-2023-3", 'employee_count': "Stripe's employee count is approximately 8,000 as of March 2023. This number is consistently reported across various business news o